In [ ]:
# Configuration - All settings at top of notebook
print("🏆 INFORMATION EXTRACTION COMPARISON: Llama 3.2 Vision vs InternVL3")
print("🎯 Focus: Information extraction performance with model-specific prompts")
print("=" * 80)

# CONFIGURATION - All settings defined here
CONFIG = {
    "model_paths": {
        "llama": "/home/jovyan/nfs_share/models/Llama-3.2-11B-Vision",
        "internvl": "/home/jovyan/nfs_share/models/InternVL3-8B"
    },
    # FIXED: Use exact proven working Llama prompt pattern from CLAUDE.md
    "llama_extraction_prompt": """<|image|>Extract data from this receipt in KEY-VALUE format.

Output format:
DATE: [date from receipt]
STORE: [store name]
GST: [GST amount]
TOTAL: [total amount]
SUBTOTAL: [subtotal amount]
ITEMS: [item names separated by |]

Extract all visible text and format as KEY: VALUE pairs only.""",
    
    # InternVL works better with YAML format
    "internvl_extraction_prompt": """<|image|>Extract key information in YAML format:

store_name: ""
date: ""
total: ""

Output only YAML. Stop after completion.""",
    
    "max_new_tokens": 64,
    "enable_quantization": True,
    "test_models": ["llama", "internvl"],
    "test_images": [
        ("image14.png", "TAX_INVOICE"),
        ("image65.png", "TAX_INVOICE"), 
        ("image71.png", "TAX_INVOICE"),
        ("image74.png", "TAX_INVOICE"),
        ("image205.png", "FUEL_RECEIPT"),
        ("image23.png", "TAX_INVOICE"),
        ("image45.png", "TAX_INVOICE"),
        ("image1.png", "BANK_STATEMENT"),
        ("image203.png", "BANK_STATEMENT"),
        ("image204.png", "FUEL_RECEIPT"),
        ("image206.png", "OTHER"),
    ]
}

print(f"✅ Configuration loaded:")
print(f"   - Models: {', '.join(CONFIG['test_models'])}")
print(f"   - Documents: {len(CONFIG['test_images'])} test images")
print(f"   - Llama prompt: Proven KEY-VALUE format (bypasses safety mode)")
print(f"   - InternVL prompt: YAML format (works best)")
print(f"   - Max tokens: {CONFIG['max_new_tokens']}")
print(f"   - Quantization: {CONFIG['enable_quantization']}")
print(f"\n📋 Ready for step-by-step information extraction comparison")

In [ ]:
# Imports and Modular Classes
import time
import torch
import json
import re
import gc
from pathlib import Path
from PIL import Image
from typing import Dict, List, Tuple, Optional, Any

class MemoryManager:
    """Memory management utilities for model testing"""
    
    @staticmethod
    def cleanup_gpu_memory():
        """Minimize memory footprint as requested"""
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
    
    @staticmethod
    def get_memory_usage() -> Dict[str, float]:
        """Get current GPU memory usage in GB"""
        if torch.cuda.is_available():
            return {
                "allocated": torch.cuda.memory_allocated() / 1024**3,
                "reserved": torch.cuda.memory_reserved() / 1024**3
            }
        return {"allocated": 0.0, "reserved": 0.0}

class UltraAggressiveRepetitionController:
    """Business document repetition detection and cleanup"""
    
    def __init__(self, word_threshold: float = 0.15, phrase_threshold: int = 2):
        self.word_threshold = word_threshold
        self.phrase_threshold = phrase_threshold
        
        # Business document specific repetition patterns
        self.toxic_patterns = [
            r"THANK YOU FOR SHOPPING WITH US[^.]*",
            r"All prices include GST where applicable[^.]*",
            r"applicable\.\s*applicable\.",
            r"GST where applicable[^.]*applicable",
            r"\\+[a-zA-Z]*\{[^}]*\}",  # LaTeX artifacts
            r"\(\s*\)",  # Empty parentheses
            r"[.-]\s*THANK YOU",
        ]
    
    def clean_response(self, response: str) -> str:
        """Clean business document extraction response"""
        if not response or len(response.strip()) == 0:
            return ""
        
        # Remove toxic business document patterns
        response = self._remove_business_patterns(response)
        
        # Remove repetitive words and phrases
        response = self._remove_word_repetition(response)
        response = self._remove_phrase_repetition(response)
        
        # Clean artifacts
        response = re.sub(r'\s+', ' ', response)
        response = re.sub(r'[.]{2,}', '.', response)
        response = re.sub(r'[!]{2,}', '!', response)
        
        return response.strip()
    
    def _remove_business_patterns(self, text: str) -> str:
        """Remove business document specific repetitive patterns"""
        for pattern in self.toxic_patterns:
            text = re.sub(pattern, "", text, flags=re.IGNORECASE)
        
        # Remove excessive "applicable" repetition
        text = re.sub(r'(applicable\.\s*){2,}', 'applicable. ', text, flags=re.IGNORECASE)
        
        return text
    
    def _remove_word_repetition(self, text: str) -> str:
        """Remove word repetition in business documents"""
        # Remove consecutive identical words
        text = re.sub(r'\b(\w+)(\s+\1){1,}', r'\1', text, flags=re.IGNORECASE)
        
        return text
    
    def _remove_phrase_repetition(self, text: str) -> str:
        """Remove phrase repetition"""
        for phrase_length in range(2, 7):
            pattern = r'\b((?:\w+\s+){' + str(phrase_length-1) + r'}\w+)(\s+\1){1,}'
            text = re.sub(pattern, r'\1', text, flags=re.IGNORECASE)
        
        return text

class KeyValueExtractionAnalyzer:
    """Analyzer for KEY-VALUE extraction results (updated from YAML to handle Llama safety mode)"""
    
    @staticmethod
    def analyze(response: str, img_name: str) -> Dict[str, Any]:
        """Analyze KEY-VALUE extraction results with consistent format"""
        response_clean = response.strip()
        
        # Detect KEY-VALUE format (both YAML and KEY: VALUE patterns)
        is_structured = bool(re.search(r'(store_name:|date:|total:|STORE:|DATE:|TOTAL:)', response_clean, re.IGNORECASE))
        
        # Extract data from both YAML and KEY-VALUE formats
        # Try YAML format first
        store_match = re.search(r'(?:store_name|STORE):\s*"?([^"\n]+)"?', response_clean, re.IGNORECASE)
        date_match = re.search(r'(?:date|DATE):\s*"?([^"\n]+)"?', response_clean, re.IGNORECASE)  
        total_match = re.search(r'(?:total|TOTAL):\s*"?([^"\n]+)"?', response_clean, re.IGNORECASE)
        
        # Fallback detection for non-structured responses
        if not store_match:
            store_match = re.search(r'(spotlight|store|business)', response_clean, re.IGNORECASE)
        if not date_match:
            date_match = re.search(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}', response_clean)
        if not total_match:
            total_match = re.search(r'(\$\d+\.\d{2}|\$\d+)', response_clean)
        
        has_store = bool(store_match)
        has_date = bool(date_match)
        has_total = bool(total_match)
        
        extraction_score = sum([has_store, has_date, has_total])
        
        return {
            "img_name": img_name,
            "response": response_clean,
            "is_structured": is_structured,
            "has_store": has_store,
            "has_date": has_date,
            "has_total": has_total,
            "extraction_score": extraction_score,
            "successful": extraction_score >= 2  # At least 2/3 fields
        }

class DatasetManager:
    """Dataset verification and management"""
    
    def __init__(self, datasets_path: str = "datasets"):
        self.datasets_path = Path(datasets_path)
    
    def verify_images(self, test_images: List[Tuple[str, str]]) -> List[Tuple[str, str]]:
        """Verify that test images exist and return verified list"""
        verified_images = []
        
        for img_name, doc_type in test_images:
            img_path = self.datasets_path / img_name
            if img_path.exists():
                verified_images.append((img_name, doc_type))
        
        return verified_images
    
    def print_verification_report(self, test_images: List[Tuple[str, str]], verified_images: List[Tuple[str, str]]):
        """Print dataset verification report"""
        print("📊 DATASET VERIFICATION")
        print("=" * 50)
        
        for img_name, doc_type in test_images:
            img_path = self.datasets_path / img_name
            if img_path.exists():
                print(f"   ✅ {img_name:<12} → {doc_type}")
            else:
                print(f"   ❌ {img_name:<12} → {doc_type} (MISSING)")
        
        print(f"\n📋 Dataset Summary:")
        print(f"   - Expected: {len(test_images)} documents")
        print(f"   - Found: {len(verified_images)} documents")
        print(f"   - Missing: {len(test_images) - len(verified_images)} documents")
        
        if len(verified_images) == 0:
            print("❌ No test images found! Check datasets/ directory")
            raise FileNotFoundError("No test images found")
        elif len(verified_images) < len(test_images):
            print("⚠️ Some test images missing but proceeding with available images")
        else:
            print("✅ All test images found")

# Initialize global utilities
memory_manager = MemoryManager()
repetition_controller = UltraAggressiveRepetitionController()
extraction_analyzer = KeyValueExtractionAnalyzer()  # Updated name
dataset_manager = DatasetManager()

print("✅ Modular classes initialized:")
print("   - MemoryManager for GPU cleanup")
print("   - UltraAggressiveRepetitionController for text cleanup")
print("   - KeyValueExtractionAnalyzer for KEY-VALUE results analysis")
print("   - DatasetManager for image verification")

In [ ]:
# Dataset Verification
# Use the modular DatasetManager class

verified_extraction_images = dataset_manager.verify_images(CONFIG["test_images"])
dataset_manager.print_verification_report(CONFIG["test_images"], verified_extraction_images)

print(f"\n🔬 KEY-VALUE Extraction Prompt:")
print(f"   {CONFIG['extraction_prompt'][:60]}...")
print(f"\n📋 Ready for model testing")

In [ ]:
# Model Loading Classes
class LlamaModelLoader:
    """Modular Llama model loader with validation"""
    
    @staticmethod
    def load_model(model_path: str, enable_quantization: bool = True):
        """Load Llama model with proper configuration"""
        from transformers import AutoProcessor, MllamaForConditionalGeneration
        from transformers import BitsAndBytesConfig
        
        processor = AutoProcessor.from_pretrained(
            model_path, trust_remote_code=True, local_files_only=True
        )
        
        model_kwargs = {
            "torch_dtype": torch.float16,
            "local_files_only": True
        }
        
        if enable_quantization:
            quantization_config = BitsAndBytesConfig(
                load_in_8bit=True,
                llm_int8_enable_fp32_cpu_offload=True,
                llm_int8_skip_modules=["vision_tower", "multi_modal_projector"],
            )
            model_kwargs["quantization_config"] = quantization_config
        
        model = MllamaForConditionalGeneration.from_pretrained(
            model_path, **model_kwargs
        ).eval()
        
        return model, processor
    
    @staticmethod
    def run_inference(model, processor, prompt: str, image, max_new_tokens: int = 64):
        """Run inference with proper device handling"""
        inputs = processor(text=prompt, images=image, return_tensors="pt")
        device = next(model.parameters()).device
        if device.type != "cpu":
            device_target = str(device).split(":")[0]
            inputs = {k: v.to(device_target) if hasattr(v, "to") else v for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                pad_token_id=processor.tokenizer.eos_token_id,
                eos_token_id=processor.tokenizer.eos_token_id,
                use_cache=True,
            )
        
        raw_response = processor.decode(
            outputs[0][inputs["input_ids"].shape[-1]:],
            skip_special_tokens=True
        )
        
        # Cleanup tensors immediately
        del inputs, outputs
        
        return raw_response

class InternVLModelLoader:
    """Modular InternVL model loader with validation"""
    
    @staticmethod
    def load_model(model_path: str, enable_quantization: bool = True):
        """Load InternVL model with proper configuration"""
        from transformers import AutoModel, AutoTokenizer
        
        tokenizer = AutoTokenizer.from_pretrained(
            model_path, trust_remote_code=True, local_files_only=True
        )
        
        model_kwargs = {
            "trust_remote_code": True,
            "torch_dtype": torch.bfloat16,
            "local_files_only": True
        }
        
        if enable_quantization:
            model_kwargs["load_in_8bit"] = True
        
        model = AutoModel.from_pretrained(
            model_path, **model_kwargs
        ).eval()
        
        return model, tokenizer
    
    @staticmethod
    def run_inference(model, tokenizer, prompt: str, image, max_new_tokens: int = 64):
        """Run inference with proper image preprocessing"""
        import torchvision.transforms as T
        from torchvision.transforms.functional import InterpolationMode
        
        transform = T.Compose([
            T.Resize((448, 448), interpolation=InterpolationMode.BICUBIC),
            T.ToTensor(),
            T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        
        pixel_values = transform(image).unsqueeze(0)
        if torch.cuda.is_available():
            pixel_values = pixel_values.cuda().to(torch.bfloat16).contiguous()
        
        raw_response = model.chat(
            tokenizer=tokenizer,
            pixel_values=pixel_values,
            question=prompt,
            generation_config={"max_new_tokens": max_new_tokens, "do_sample": False}
        )
        
        if isinstance(raw_response, tuple):
            raw_response = raw_response[0]
        
        # Cleanup tensors immediately
        del pixel_values
        
        return raw_response

def get_model_prompt(model_name: str, config: Dict) -> str:
    """Get the appropriate prompt for each model"""
    if model_name.lower() == "llama":
        return config["llama_extraction_prompt"]
    elif model_name.lower() == "internvl":
        return config["internvl_extraction_prompt"]
    else:
        # Fallback to llama prompt
        return config["llama_extraction_prompt"]

def validate_model(model_loader_class, model_path: str, config: Dict, model_name: str) -> Tuple[bool, Optional[Any], Optional[Any], float]:
    """STEP 1: LOAD MODEL FIRST - separate from prompt testing"""
    memory_manager.cleanup_gpu_memory()
    model_start_time = time.time()
    
    try:
        print(f"🔄 STEP 1: Loading model from {model_path}...")
        
        # LOAD MODEL FIRST - no prompts yet
        model, processor_or_tokenizer = model_loader_class.load_model(
            model_path, config["enable_quantization"]
        )
        
        model_load_time = time.time() - model_start_time
        print(f"✅ Model loaded successfully in {model_load_time:.1f}s")
        
        # STEP 2: Simple validation that model can run basic inference
        print(f"🔍 STEP 2: Testing basic model functionality...")
        img_path = dataset_manager.datasets_path / "image14.png"
        
        if not img_path.exists():
            print(f"❌ Test image not found: {img_path}")
            del model, processor_or_tokenizer
            memory_manager.cleanup_gpu_memory()
            return False, None, None, model_load_time
        
        image = Image.open(img_path).convert("RGB")
        
        # Use the simplest possible prompt to test model loading (not extraction quality)
        simple_test_prompt = "<|image|>What do you see?"
        
        try:
            raw_response = model_loader_class.run_inference(
                model, processor_or_tokenizer, simple_test_prompt, 
                image, 32  # Short response for validation
            )
            
            # MODEL VALIDATION: Just check that inference works
            if raw_response and len(raw_response.strip()) > 0:
                print(f"✅ Model validation passed - inference works")
                print(f"   Test response: {raw_response[:50]}...")
                return True, model, processor_or_tokenizer, model_load_time
            else:
                print(f"❌ Model validation failed - no response")
                del model, processor_or_tokenizer
                memory_manager.cleanup_gpu_memory()
                return False, None, None, model_load_time
        
        except Exception as inference_error:
            print(f"❌ Model validation failed - inference error: {str(inference_error)[:100]}...")
            del model, processor_or_tokenizer
            memory_manager.cleanup_gpu_memory()
            return False, None, None, model_load_time
            
    except Exception as e:
        print(f"❌ Model loading failed: {str(e)[:100]}...")
        memory_manager.cleanup_gpu_memory()
        return False, None, None, 0.0

print("✅ Model loader classes defined:")
print("   - LlamaModelLoader with validation")
print("   - InternVLModelLoader with validation") 
print("   - get_model_prompt() - Returns model-specific prompts")
print("   - validate_model() - STEP 1: Load model, STEP 2: Test basic inference")
print("   - SEPARATED: Model loading from prompt application")

In [ ]:
# Test Llama Model
print("🔬 TESTING LLAMA MODEL")
print("=" * 50)

# Initialize results storage
extraction_results = {
    "llama": {"documents": [], "successful": 0, "total_time": 0},
    "internvl": {"documents": [], "successful": 0, "total_time": 0}
}

# Use modular validation function with model name
llama_valid, llama_model, llama_processor, llama_load_time = validate_model(
    LlamaModelLoader, 
    CONFIG["model_paths"]["llama"], 
    CONFIG,
    "llama"
)

if llama_valid:
    print("✅ Llama model ready for full testing")
    print(f"🔬 Using Llama-specific prompt: {CONFIG['llama_extraction_prompt'][:50]}...")
    # Store for next cell
    globals()['llama_model'] = llama_model
    globals()['llama_processor'] = llama_processor
    globals()['llama_load_time'] = llama_load_time
else:
    print("❌ Llama model validation failed")

In [ ]:
# Test InternVL Model  
print("🔬 TESTING INTERNVL MODEL")
print("=" * 50)

# Use modular validation function with model name
internvl_valid, internvl_model, internvl_tokenizer, internvl_load_time = validate_model(
    InternVLModelLoader,
    CONFIG["model_paths"]["internvl"],
    CONFIG,
    "internvl"
)

if internvl_valid:
    print("✅ InternVL model ready for full testing")
    print(f"🔬 Using InternVL-specific prompt: {CONFIG['internvl_extraction_prompt'][:50]}...")
    # Store for next cell
    globals()['internvl_model'] = internvl_model
    globals()['internvl_tokenizer'] = internvl_tokenizer
    globals()['internvl_load_time'] = internvl_load_time
else:
    print("❌ InternVL model validation failed")

In [ ]:
# Run Full Extraction Test - Llama
if 'llama_model' in globals() and llama_model is not None:
    print("🔍 FULL EXTRACTION TEST - LLAMA")
    print("=" * 50)
    print(f"🎯 Using PROVEN Llama prompt pattern from CLAUDE.md")
    
    # Get Llama-specific prompt
    llama_prompt = get_model_prompt("llama", CONFIG)
    
    total_inference_time = 0
    
    for i, (img_name, doc_type) in enumerate(verified_extraction_images, 1):
        try:
            img_path = dataset_manager.datasets_path / img_name
            image = Image.open(img_path).convert("RGB")
            
            inference_start = time.time()
            
            # Use Llama-specific prompt
            raw_response = LlamaModelLoader.run_inference(
                llama_model, llama_processor, llama_prompt,
                image, CONFIG["max_new_tokens"]
            )
            
            inference_time = time.time() - inference_start
            total_inference_time += inference_time
            
            cleaned_response = repetition_controller.clean_response(raw_response)
            analysis = extraction_analyzer.analyze(cleaned_response, img_name)
            analysis["inference_time"] = inference_time
            analysis["doc_type"] = doc_type
            
            extraction_results["llama"]["documents"].append(analysis)
            
            if analysis["successful"]:
                extraction_results["llama"]["successful"] += 1
            
            # Consistent output format as requested
            status = "✅" if analysis["successful"] else "❌"
            structured_status = "S" if analysis["is_structured"] else "T"
            print(f"   {i:2d}. {img_name:<12} {status} {inference_time:.1f}s | {structured_status} | {analysis['extraction_score']}/3")
            
            # Immediate tensor cleanup - minimizing memory footprint
            del image
            
            # Periodic GPU cleanup every 3 images
            if i % 3 == 0:
                memory_manager.cleanup_gpu_memory()
            
        except Exception as e:
            print(f"   {i:2d}. {img_name:<12} ❌ Error: {str(e)[:30]}...")
    
    extraction_results["llama"]["total_time"] = total_inference_time
    extraction_results["llama"]["avg_time"] = total_inference_time / len(verified_extraction_images)
    
    print(f"\n📊 Llama Results:")
    print(f"   Success rate: {extraction_results['llama']['successful']}/{len(verified_extraction_images)}")
    print(f"   Average time: {extraction_results['llama']['avg_time']:.1f}s per document")
    
    # Cleanup Llama model to free memory for InternVL
    del llama_model, llama_processor
    memory_manager.cleanup_gpu_memory()
    
else:
    print("⚠️ Llama model not available - skipping full test")

In [ ]:
# Run Full Extraction Test - InternVL
if 'internvl_model' in globals() and internvl_model is not None:
    print("🔍 FULL EXTRACTION TEST - INTERNVL")
    print("=" * 50)
    print(f"🎯 Using YAML format prompt (works best for InternVL)")
    
    # Get InternVL-specific prompt
    internvl_prompt = get_model_prompt("internvl", CONFIG)
    
    total_inference_time = 0
    
    for i, (img_name, doc_type) in enumerate(verified_extraction_images, 1):
        try:
            img_path = dataset_manager.datasets_path / img_name
            image = Image.open(img_path).convert("RGB")
            
            inference_start = time.time()
            
            # Use InternVL-specific prompt
            raw_response = InternVLModelLoader.run_inference(
                internvl_model, internvl_tokenizer, internvl_prompt,
                image, CONFIG["max_new_tokens"]
            )
            
            inference_time = time.time() - inference_start
            total_inference_time += inference_time
            
            cleaned_response = repetition_controller.clean_response(raw_response)
            analysis = extraction_analyzer.analyze(cleaned_response, img_name)
            analysis["inference_time"] = inference_time
            analysis["doc_type"] = doc_type
            
            extraction_results["internvl"]["documents"].append(analysis)
            
            if analysis["successful"]:
                extraction_results["internvl"]["successful"] += 1
            
            # Consistent output format as requested
            status = "✅" if analysis["successful"] else "❌"
            structured_status = "S" if analysis["is_structured"] else "T"
            print(f"   {i:2d}. {img_name:<12} {status} {inference_time:.1f}s | {structured_status} | {analysis['extraction_score']}/3")
            
            # Immediate tensor cleanup - minimizing memory footprint
            del image
            
            # Periodic GPU cleanup every 3 images
            if i % 3 == 0:
                memory_manager.cleanup_gpu_memory()
            
        except Exception as e:
            print(f"   {i:2d}. {img_name:<12} ❌ Error: {str(e)[:30]}...")
    
    extraction_results["internvl"]["total_time"] = total_inference_time
    extraction_results["internvl"]["avg_time"] = total_inference_time / len(verified_extraction_images)
    
    print(f"\n📊 InternVL Results:")
    print(f"   Success rate: {extraction_results['internvl']['successful']}/{len(verified_extraction_images)}")
    print(f"   Average time: {extraction_results['internvl']['avg_time']:.1f}s per document")
    
    # Cleanup InternVL model 
    del internvl_model, internvl_tokenizer
    memory_manager.cleanup_gpu_memory()
    
else:
    print("⚠️ InternVL model not available - skipping full test")

In [ ]:
# Final Comparison and Recommendation
class ResultsAnalyzer:
    """Modular results analysis and comparison"""
    
    @staticmethod
    def print_final_comparison(extraction_results: Dict, verified_images: List):
        """Print final comparison between models"""
        print(f"\n{'=' * 80}")
        print("🏆 FINAL RECOMMENDATION: BEST MODEL FOR INFORMATION EXTRACTION")
        print(f"{'=' * 80}")
        
        # Compare both models' performance
        llama_success = 0
        llama_total = 0
        llama_avg_time = 0
        internvl_success = 0
        internvl_total = 0
        internvl_avg_time = 0
        
        if extraction_results["llama"]["documents"]:
            llama_total = len(extraction_results["llama"]["documents"])
            llama_success = extraction_results["llama"]["successful"]
            llama_avg_time = extraction_results["llama"]["avg_time"]
        
        if extraction_results["internvl"]["documents"]:
            internvl_total = len(extraction_results["internvl"]["documents"])
            internvl_success = extraction_results["internvl"]["successful"]
            internvl_avg_time = extraction_results["internvl"]["avg_time"]
        
        print(f"📊 INFORMATION EXTRACTION COMPARISON:")
        print(f"{'Model':<12} {'Success Rate':<15} {'Avg Time':<12} {'Best For'}")
        print("-" * 60)
        
        if llama_total > 0:
            llama_rate = llama_success / llama_total * 100
            print(f"{'LLAMA':<12} {llama_rate:.1f}% ({llama_success}/{llama_total}){'':<5} {llama_avg_time:.1f}s{'':<7} Large context")
        
        if internvl_total > 0:
            internvl_rate = internvl_success / internvl_total * 100
            print(f"{'INTERNVL':<12} {internvl_rate:.1f}% ({internvl_success}/{internvl_total}){'':<5} {internvl_avg_time:.1f}s{'':<7} Production speed")
        
        # Make recommendation
        if internvl_total > 0 and llama_total > 0:
            internvl_rate = internvl_success / internvl_total * 100
            llama_rate = llama_success / llama_total * 100
            
            if internvl_rate > llama_rate:
                recommended = "INTERNVL"
                reason = f"Higher success rate ({internvl_rate:.1f}% vs {llama_rate:.1f}%) and faster inference"
            elif llama_rate > internvl_rate:
                recommended = "LLAMA"
                reason = f"Higher success rate ({llama_rate:.1f}% vs {internvl_rate:.1f}%)"
            else:
                recommended = "INTERNVL"
                reason = f"Equal success rate but {internvl_avg_time/llama_avg_time:.1f}x faster inference"
            
            print(f"\n🥇 RECOMMENDED FOR INFORMATION EXTRACTION: {recommended}")
            print(f"   Reason: {reason}")
            print(f"   Use case: Business document processing (receipts, invoices, statements)")
        elif internvl_total > 0:
            print(f"\n🥇 RECOMMENDED: INTERNVL (only model tested successfully)")
        elif llama_total > 0:
            print(f"\n🥇 RECOMMENDED: LLAMA (only model tested successfully)")
        else:
            print(f"\n⚠️ No successful tests - investigate model loading issues")
        
        print(f"\n✅ COMPLETE: Information extraction performance comparison finished!")
        print(f"📋 This answers the user's question about best model for their information extraction job")

# Use the modular analyzer
results_analyzer = ResultsAnalyzer()
results_analyzer.print_final_comparison(extraction_results, verified_extraction_images)

# Show the model-specific prompts being used
print(f"\n🔬 MODEL-SPECIFIC PROMPTS USED:")
print(f"{'='*50}")
print(f"🔥 LLAMA PROMPT (Proven pattern from CLAUDE.md):")
print(CONFIG["llama_extraction_prompt"])
print(f"\n{'='*50}")
print(f"🎯 INTERNVL PROMPT (YAML format):")
print(CONFIG["internvl_extraction_prompt"])
print(f"{'='*50}")
print(f"✅ Confirmed: Using model-optimized prompts for maximum performance")